# Project Overview: Dynamic Bar Chart for Toronto Ward Budgets (Year 2010 to Year 2024)

## Prerequisites
- **Python**
- **Data Visualization**
- **Data Analysis**
- **Automation**
- **Google Chrome**

## 1. Data Source and Description

### 1.1 Data Source
The data used in this project is **automatically** retrieved and downloaded by Python from various Excel files available at the following link. [Toronto Open Data](https://open.toronto.ca/dataset/budget-capital-budget-plan-by-ward-10-yr-approved/
). The downloaded data is stored in the data folder within the current year's work directory.

<img src="data files.png" alt="data files" width="400" height="400">


### 1.2 Data Description

Each file contains financial data for **44 (25 from 2019)** wards in Toronto, covering the time span **from 2010 to 2024**. The columns detail annual budgets, and key columns of interest are:

- `Ward/Project Number`: Identifier for each ward and project.
- `Year` columns: Columns that represent different years' budget data.

## 2. Project Objective (Data Visualization)

The primary objective of this project is to create a **Dynamic Bar Chart** that compares the annual budgets of different wards in Toronto. This comparison will help visualize how each ward's budget changes over time and highlight disparities and trends across the city. Additionally, **animated visuals** are incorporated to enhance the dynamic presentation of the data.

### 2.1 Dynamic Bar Chart Features
- **Vertical Comparison**: Compare the budget changes of the same ward across different years, showing how the budget evolves over time for that specific ward.
- **Horizontal Comparison**: Compare the budgets of different wards for the same year, highlighting how budgets vary among wards in a given year.

<img src="viz_by_python.gif" alt="Dynamic Bar Chart" width="400" height="400">

## 3. Data Processing 

### 3.1 Reading Excel Files

### 3.2 Extracting and Cleaning Data (ETL)

Over the past 14 years, the format of the financial data tables has been updated multiple times, with key columns, their names, and positions changing frequently. This requires Python to dynamically locate and extract the necessary data. Key challenges include:

- Conversion from XLS to XLSX formats
- Splitting merged cells
- Adjusting column positions and names due to format changes and the reorganization of ward divisions

Below are examples of different formats encountered:

<img src="format1.png" alt="format1" width="400" height="200"> 
<img src="format2.png" alt="format2" width="400" height="200">
<img src="format3.png" alt="format3" width="400" height="200">


## 4. Data Calculating for Visualization 

Prepare the data for visualization by:

- Generating mappings for ward number and ward name.
- Adding new columns as needed.
- Filling missing values with fillna.
- Ensuring data accuracy for accurate visualizations.

## 5. Data Visualization (Dynamic Bar Chart)
   - Use Plotly to create an interactive bar chart.
   - Display annual budgets for each ward, animating the chart to show how budgets change both from year to year and across different wards.
   

(End)


### 0. Install Python Modules

In [578]:
import subprocess
import sys

# 0.1 The following function installs/tests the major required packages for this script

def install_and_import(package):
    try:
        import_name = package
        __import__(import_name)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully.")
    finally:
        globals()[package] = __import__(import_name)


import_packages = ['selenium', 'webdriver_manager', 'xlrd', 'openpyxl', 'xls2xlsx']

for package in import_packages:
    install_and_import(package)
print('Packages have been installed.')

Packages have been installed.


### 1. Import Libraries and Define Variables

In [579]:
# Get download links
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

# Download files
import requests
import os

# Data ETL
import pandas as pd
import openpyxl
from openpyxl import Workbook, load_workbook
from xls2xlsx import XLS2XLSX

# Data Calculating
from sklearn.preprocessing import MinMaxScaler

# Suppress specific warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data Visualization
import plotly.express as px


source_data_link         = 'https://open.toronto.ca/dataset/budget-capital-budget-plan-by-ward-10-yr-approved/'
source_button_XPATH      = '//*[@id="header-Download"]/div/h3/button'
link_XPATH               = '//*[@id="table-resources"]'
download_folder          = 'data'
ward_column_list         = ['Ward', 'Ward/Project Number', 'Ward/Project Num.', 'Ward Name']
inserted_column          = 'Year'
rename_column_1          = 'Budget_Year_1'
rename_column_2          = 'Budget_Year_2'
extracted_data_file      = 'budget_data.csv'
report_data_file         = 'report_data.csv'
family = 'sans-serif'
purple_color = '#544698'

### 2. Data Processing
 
- **2.1 Function to get download links for multiple Excel files**

**_Please note that this code will automatically use Google Chrome to search for download links. If you do not have the Google Chrome browser installed, please manually download the data from the source data link (in the previous cell) to the 'data' folder in your current working directory._**

In [580]:
def get_links(url, button_XPATH, links_area_XPATH):

    # Set up Selenium options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (no browser window)

    # Initialize the WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Open the target URL
    driver.get(url)

    # Wait for the button to be clickable and click it
    time.sleep(1)  # Adjust wait time as needed
    button = driver.find_element(By.XPATH, button_XPATH)  # Adjust XPath as needed
    button.click()

    # Wait for the content to load
    time.sleep(1)  # Adjust wait time as needed

    # Find the specific area containing the links
    links_area = driver.find_element(By.XPATH, links_area_XPATH)

    # Find all links within the specified area
    links = links_area.find_elements(By.TAG_NAME, 'a')

    # Extract all download links (including .xls and .xlsx) that contain "download"
    download_links = [
        link.get_attribute('href') for link in links
        if 'download' in link.get_attribute('href').lower() and re.search(r'\.(xls|xlsx)$', link.get_attribute('href'), re.IGNORECASE)
    ]

    # Close the browser
    driver.quit()
    
    return download_links

- **2.2 Function to download Files using the download links**

In [581]:
def download_files(download_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    for link in download_links:
        # Extract the file name from the link
        file_name = os.path.join(download_folder, link.split('/')[-1])

        # Send a GET request to download the file
        response = requests.get(link)
        response.raise_for_status()  # Ensure we notice bad responses

        # Write the file to the specified folder
        with open(file_name, 'wb') as file:
            file.write(response.content)

        print(f"Downloaded {file_name}")


- **2.3 Data ETL (Extraction, Transformation, Loading)**

- 2.3.1 Subfunction 1 to get the year column

In [582]:
def get_year_from_column(name):
    # Function to check if a column name can be converted to an int
    try:
        return int(name)
    except:        
        return None

- 2.3.2 Subfunction 2 to convert .xls to .xlsx

In [583]:
def convert_xls_to_xlsx(xls_path, xlsx_path):
    
    xls_file = XLS2XLSX(xls_path)
    xls_file.to_xlsx(xlsx_path)
    

- 2.3.3 Subfunction 3 to fill the unmerged cells

In [584]:
def fill_unmerged_cells(file_path):
    workbook = load_workbook(filename=file_path)

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        for merge in list(sheet.merged_cells.ranges):
            min_col, min_row, max_col, max_row = merge.bounds
            top_left_cell_value = sheet.cell(row=min_row, column=min_col).value
            sheet.unmerge_cells(str(merge))
            for row in range(min_row, max_row + 1):
                for col in range(min_col, max_col + 1):
                    sheet.cell(row=row, column=col).value = top_left_cell_value
    workbook.save(file_path)
    print(f"Unmerged cells have been saved for Year: {(os.path.basename(file_path))[0:4]}")  
    

- 2.3.4 Subfunction 4 to convert string to numerical values

In [585]:
def to_numeric(value):
    try:
        return float(value)
    except ValueError:
        return 0

- 2.3.5 Subfunction 5 to process the data

In [586]:
def data_processing(file_path, ward_column_list, sheet_name=None, inserted_column = None, rename_column_1 = None, rename_column_2 = None ):
    file_name = os.path.basename(file_path)
    year_number = file_name[0:4]
    filtered_data = None
    # 1) Read the Excel file and convert the data to string temporarily
    if sheet_name:
        try: 
            # Read a specific sheet into a DataFrame
            excel_data = pd.read_excel(file_path, header=None, sheet_name=sheet_name, dtype=str,engine='openpyxl')
        except: 
            excel_data = pd.read_excel(file_path, header=None, sheet_name=sheet_name, dtype=str,engine='xlrd')
        # Wrap DataFrame in a dictionary to handle uniformly
        excel_data = {sheet_name: excel_data}
    else:
        try:
            # Read all sheets into a dictionary
            excel_data = pd.read_excel(file_path, header=None, dtype=str, sheet_name=None, engine='openpyxl')
        except:
            excel_data = pd.read_excel(file_path, header=None, dtype=str, sheet_name=None, engine='xlrd')
    # 2) Iterate over the sheets and find the actual header
    for sheet_name, sheet_data in excel_data.items():
        # Flatten the DataFrame to a Series for easier searching
        # flat_series = sheet_data.stack().astype(str)
        
        ward_row_index = None
        # Check if 'ward_column' is found in any cell
        for ward_column in ward_column_list:
            # print(f'Searching for Ward_column : {ward_column}')
            # if flat_series.str.contains(ward_column, case=False, na=False).any():

            # Find all the index of the row containing 'Ward/Project Number' or 'Ward'
            matched_rows = sheet_data[sheet_data.apply(lambda row: row.astype(str).str.strip() == ward_column).any(axis=1)]
            if len(matched_rows) == 0:
                # The result contains nothing
                continue
            elif len(matched_rows) ==1 and int(year_number)<2019:
                ward_row_index = matched_rows.index[0]
            else:
                # Multiple values have been found                                  
                # ward_column =='Ward' 
                # print(matched_rows)
                for index, row in matched_rows.iterrows():
                    # print(f'Index is "{index}", Row is "{row}"')
                    # Check if the column to the right of 'Ward' contains 'Project Name'
                    ward_col_index = row[row.astype(str).str.strip() == ward_column].index[0]
                    # print(f"Ward_col_index is {ward_col_index}")
                    # if ward_col_index + 1 < len(sheet_data.columns):
                    #     # print(len(row))
                    #     # print(type(row))
                    project_col = row[row.astype(str).str.strip() == 'Project Name']
                    # print(f"Project_col is {project_col}")
                    if project_col.empty:
                        continue
                    else:
                        if int(year_number) >=2019:
                            ward_row_index = index
                        
                        elif int(year_number) == 2010:
                            project_col_index = project_col.index[0]
                            if  ward_col_index+1 == project_col_index :
                                ward_column = row[ward_col_index]
                                # 'Ward' is a desired column name
                                ward_row_index = index
                            break
                 
            if ward_row_index >=0:
                # Ward Column is found    
                print(f"'{ward_column}' found in the sheet '{sheet_name}'.")
                above_budget_cell_value = None
                row_values = sheet_data.iloc[ward_row_index].tolist()
               
                if 'Budget' in row_values:
                    # Some files contain the current year number above the "Budget" cell
                    budget_cell_index = row_values.index('Budget')

                    # Ensure the index is not None:
                    if budget_cell_index > 0:
                        above_budget_cell_value = sheet_data.iloc[ward_row_index - 1, budget_cell_index]
                        
                # 4) Remove all rows above the 'Ward/Project Number' row and reset the index
                if ward_row_index >0 :
                    data = sheet_data.iloc[ward_row_index:].reset_index(drop=True)
                else:
                    data = sheet_data.reset_index(drop=True)
                # 5) Set the first row of data as column headers
                new_header = data.iloc[0]
                data = data[1:]
                data.columns = new_header
                if above_budget_cell_value:
                    data.rename(columns={'Budget': above_budget_cell_value}, inplace=True)
                
                # 6) Extract all column names and find columns that can be converted to year
                year_columns = [(col, get_year_from_column(col)) for col in data.columns]
                year_columns = [col for col, year in year_columns if year is not None]

                # 7) Get the first two year columns
                first_year_column, second_year_column = year_columns[:2]
                    
                # 8) Keep only necessary columns
                if int(year_number)>=2019:
                    selected_columns = [ward_column, first_year_column, second_year_column, 'Ward Number']
                else:
                    selected_columns =[ward_column, first_year_column, second_year_column]
                selected_data = data[selected_columns]
                    
                # 9) Filter the data to keep only rows where 'Ward/Project Number' ends with 'Total'
                selected_data.loc[:,ward_column] = selected_data[ward_column].fillna(' ')
                # print(selected_data['Ward Number'].unique())
                # print(year_number)
                if year_number == '2010':
                    filtered_data = selected_data[selected_data[ward_column].str.startswith('Ward')]
                    # print(f'filtered_data is {filtered_data}')
                    filtered_data = filtered_data.dropna(subset=[first_year_column, second_year_column])
                elif int(year_number) >=2019:
                    # As Ward Number has chosen, those indicating non-numerical value will be removed
                    selected_data = selected_data[selected_data['Ward Number'].apply(lambda x: str(x).isdigit())]
                    # print(selected_data.head(5))
                    original_data= selected_data.copy()

                    original_data[first_year_column]= original_data[first_year_column].fillna(0)
                    original_data[second_year_column]= original_data[second_year_column].fillna(0)

                    original_data[first_year_column] = original_data[first_year_column].apply(to_numeric)
                    original_data[second_year_column] = original_data[second_year_column].apply(to_numeric)
                    # print(original_data['Ward Number'].unique())

                    # List of columns to sum
                    columns_to_sum = [first_year_column, second_year_column]  # replace with your actual column names
                    # Perform the sum operation and keep all other columns
                    original_data[columns_to_sum] = original_data.groupby('Ward Number')[columns_to_sum].transform('sum')
                    filtered_data = original_data
                    # filtered_data= original_data.groupby('Ward Number').sum().reset_index()\
                    # Try to avoid using groupby this way, the non-numerical columns will be grouped as well
                    filtered_data = filtered_data.rename(columns={
                        'Ward Number':'Ward_Number'
                    })
                    # print(filtered_data)
                else:
                    filtered_data = selected_data[selected_data[ward_column].str.endswith('Total')]

                # 10) Insert 'Year' column and reorder the columns
                if inserted_column is not None:
                    filtered_data = filtered_data.copy()
                    filtered_data[inserted_column]= first_year_column
                    if int(year_number)>=2019:
                        ordered_columns =[ward_column, inserted_column, first_year_column, second_year_column, 'Ward_Number']
                    else:
                        ordered_columns =[ward_column, inserted_column, first_year_column, second_year_column]
                    filtered_data = filtered_data[ordered_columns]
                    filtered_data = filtered_data.rename(columns={
                        ward_column: ward_column_list[0],
                        first_year_column: rename_column_1,
                        second_year_column : rename_column_2
                    })
                # 11) Drop duplicates and null values
                filtered_data = filtered_data.drop_duplicates()
                filtered_data = filtered_data.dropna()
    # print(filtered_data)    
    return filtered_data

- 2.3.6 Function for Data ETL

In [587]:
def data_etl(download_folder, ward_column_list, sheet_name=None, inserted_column = None, rename_column_1 = None, rename_column_2 = None ):    
    
    # 0) Initialize an empty DataFrame for combining data
    column_list =[ward_column_list[0], inserted_column, rename_column_1, rename_column_2]
    budget_df = pd.DataFrame(columns=column_list)

    # 1) Get the absolute path of the download folder
    folder_path = os.path.abspath(download_folder)

    # 2) Iterate over all Excel files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            year_number = file_name[0:4]
            if '20' in file_name:
                
                file_path = os.path.join(folder_path, file_name)
                if file_name.endswith('.xls') and '2010' in file_name:
                    # !This is very important! The merged cells in the Excel must be unmerged and filled before loading the values into a dataframe
                    xlsx_path = file_path.replace('.xls','.xlsx')
                    # print(file_name)
                    convert_xls_to_xlsx(file_path,xlsx_path)
                    fill_unmerged_cells(xlsx_path)
                    continue
                
                print(f'Extracting data for Year "{file_name[0:4]}"....')
                sheet_names = pd.ExcelFile(file_path).sheet_names
                filtered_data = data_processing(file_path, ward_column_list, sheet_names[0], inserted_column, rename_column_1, rename_column_2)
                if filtered_data is not None and not filtered_data.empty:
                    budget_df = budget_df.copy()
                    budget_df = pd.concat([budget_df, filtered_data], ignore_index=True)
                    print('Budget Data has been found')
                else:
                    print('Budget Data not found.')
    # print(budget_df)
    return budget_df
            

### 3. Data Calculating for Reporting (Visualization)

- **3.1 Subfunction 1 to generate the ward mapping (ward number : ward name)**

In [588]:
def get_ward_mapping(extracted_data_file, ward_column_list, year_number=2010):   
    # Read csv file
    extracted_budget_data = pd.read_csv(extracted_data_file)
    budget_year_number = extracted_budget_data[extracted_budget_data['Year'] == year_number]

    # Get ward information
    ward_info = budget_year_number[ward_column_list[0]].unique()
    
    # Generate ward info mapping
    ward_mapping = {}
    for ward in ward_info:
        # print(ward)
        match = re.match(r'Ward\s+(\d+)\s+(.*)', ward)
        if match:
            number = match.group(1)
            name = match.group(2).strip()
            # Format the ward number to be 2-digit
            formatted_number = f"{int(number):02d}"
            # Generate mappings
            ward_mapping[formatted_number] = name
            # print(f"Parsed '{ward}' as number: {formatted_number}, name: {name}")
        else:
            print(f"Failed to parse '{ward}'")

    # Print out the result
    # print(ward_mapping)
    return ward_mapping
# ward_mapping = get_ward_mapping(extracted_data_file, ward_column_list)
# ward_mapping

- **3.2 Subfunction 2 to extract Ward Number as a new column**

In [589]:
def extract_and_format_number(ward):
    matches = re.findall(r'\d+', ward)
    if matches:
        number = matches[0]
        # Format number to be 2-digit
        formatted_number = f"{int(number):02d}"
        return formatted_number
    return None

- **3.3 Function to calculate the extracted data**

In [590]:
def data_calculating(extracted_data_file, ward_mapping, ward_column_list, scale = False):
    budget_data = pd.read_csv(extracted_data_file)   
    # Apply the function only to rows where Year is less than 2019
    mask = budget_data['Year'] < 2019
    budget_data.loc[mask, 'Ward_Number'] = budget_data.loc[mask, ward_column_list[0]].apply(extract_and_format_number)
    # budget_data['Ward_Number']= budget_data[ward_column_list[0]].apply(extract_and_format_number)
    
    budget_data.loc[mask,'Ward_Name'] = budget_data.loc[mask,'Ward_Number'].map(ward_mapping)

    # For rows where Year is 2019 or later, directly copy the values
    mask_from_2019 = budget_data['Year'] >= 2019
    # print(budget_data)
    budget_data.loc[mask_from_2019, 'Ward_Name'] = budget_data.loc[mask_from_2019, ward_column_list[0]]

    budget_data = budget_data[budget_data[ward_column_list[0]]!='DO NOT USE']
    budget_data[rename_column_1] = budget_data[rename_column_1].apply(to_numeric)
    budget_data[rename_column_2] = budget_data[rename_column_2].apply(to_numeric)

    budget_data_sorted = budget_data.sort_values(by=['Year', 'Ward_Number'])

    if scale:
        # Initialize MinMaxScaler
        scaler = MinMaxScaler()

        # Scale the specified columns to the range [0, 1]
        for column in [rename_column_1, rename_column_1]:
            if column in budget_data_sorted.columns:
                budget_data_sorted[column] = scaler.fit_transform(budget_data_sorted[[column]])
    
    budget_data_sorted = budget_data_sorted.dropna()
    budget_data_sorted = budget_data_sorted.drop_duplicates()
    # print(budget_data_sorted)
    return budget_data_sorted


### 4. Data Visualization

- **4.1 Create dynamic and interative bar chart using Plotly.** 

In [591]:
def data_visualizing(report_data_file,rename_column_1, purple_color= purple_color, family= family):
    report_data = pd.read_csv(report_data_file)

    # Convert the negative values into small positive values 
    report_data[rename_column_1] = report_data[rename_column_1].apply(lambda x: x if x > 0 else 0.1)

    fig = px.bar(report_data,
                y="Ward_Number",
                x =rename_column_1,
                title="Toronto Budgets By Ward (Year 2010 to Year 2024)",

                labels={'Ward_Name': 'Ward', 'Budget_Year_1': 'Budget'},
                text= 'Ward_Name',
                color = rename_column_1,
                color_continuous_scale='Sunset',  
                log_x=True,
                orientation='h',
                animation_frame='Year',  
                range_x=[0.01, report_data[rename_column_1].max() * 1.1] 
                )

    fig.update_traces(
        texttemplate='%{text}',  
        textposition='inside',  
        insidetextanchor='end',  
        textfont_size=12,  
    )
    fig.update_layout(
        width=900, 
        height=900,  
        yaxis=dict(tickmode='linear'),  
        transition={'duration': 1000},  
        xaxis_title="Budget In $1000",  
        title_font = dict(size = 18, color = purple_color, family = family),
    )

    fig.update_layout(
        annotations=[
            dict(
                x=1, y=1.06,
                xref='paper', yref='paper',
                text='Julia Ren, https://github.com/renrihui8415/',
                showarrow=False,
                font=dict(size=10, color=purple_color),
                align='right',
                xanchor='right',
                yanchor='top'
            ),
            dict(
                x=0, y=1.02,
                xref='paper', yref='paper',
                text='The visualization shows how annual budgets for Toronto''s wards, which decreased from 44 to 25 after 2019, have evolved from 2010 to 2024, with each bar representing a ward.',
                showarrow=False,
                font=dict(size=10, color=purple_color),
                align='left',
                xanchor='left',
                yanchor='top'
            ),
            dict(
                x=0, y=1.04,
                xref='paper', yref='paper',
                text='Data Source: [Toronto Open Data](https://open.toronto.ca/dataset/budget-capital-budget-plan-by-ward-10-yr-approved/)',
                showarrow=False,
                font=dict(size=10, color=purple_color),
                align='left',
                xanchor='left',
                yanchor='top'
            ),

            dict(
                x=1, y=1.08,
                xref='paper', yref='paper',
                text='Date: August 2024',
                showarrow=False,
                font=dict(size=10, color=purple_color),
                align='right'
            ),
        ]
    )

    # Control speed
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2000  
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 2000  
    fig.update_layout(autosize=True)

    fig.show()


### 5. Main() Function to streamline and automate data visualization, data processing and data analysis.

In [592]:
def main():

    # 1. Get the download links from data source
    download_links = get_links(source_data_link, source_button_XPATH, link_XPATH)

    # Print all download links
    # print("Download links:")
    # for link in download_links:
    #     print(link)
    print("Step 1. Download links have been obtained.")

    # 2. Download files using the download links
    download_files(download_links, download_folder)
    print(f"Step 2. Data files have been download in '{os.path.abspath(download_folder)}'. ")
    
    # 3. Data Process (ETL)
    budget_data = data_etl(download_folder, ward_column_list, None, inserted_column, rename_column_1, rename_column_2 )
    if budget_data.empty:
        return "Budget_data not found in the entire folder."
    # print(budget_data.head(5))
    
    # Save the extracted data into .csv file
    budget_data.to_csv(extracted_data_file)
    print("Step 3. Budget data has been extracted and saved.")

    # 4. Data Calculation for Reporting(Visualization)
    ward_mapping = get_ward_mapping(extracted_data_file,ward_column_list)
    report_data = data_calculating(extracted_data_file, ward_mapping, ward_column_list, scale = False)
    print('Step 4. Report data have been calculated and saved.')
    # print(report_data.head(5))
    report_data.to_csv(report_data_file)
    
    # 5. Data Visualization
    print('Step 5. Dynamic Bar Plot has been generated.')
    print('Upon first playback, there may be a 1-second delay.')
    data_visualizing(report_data_file,rename_column_1, purple_color= purple_color, family= family)
    
if __name__ == "__main__":
    main()

Step 1. Download links have been obtained.
Downloaded data/2010-to-2019-city-program-by-ward.xls
Downloaded data/2011-to-2020-capital-budget-plan-by-ward.xls
Downloaded data/2012-to-2021-city-program-by-ward.xlsx
Downloaded data/2013-to-2022-city-program-by-ward.xlsx
Downloaded data/2014-to-2023-capital-budget-plan-by-ward.xlsx
Downloaded data/2015-to-2024-capital-budget-plan-by-ward.xls
Downloaded data/2016-to-2025-capital-budget-plan-by-ward.xls
Downloaded data/2017-to-2026-capital-budget-plan-by-ward.xlsx
Downloaded data/2018-to-2027-capital-budget-plan-by-ward.xlsx
Downloaded data/2018-to-2027-capital-budget-plan-by-program.xlsx
Downloaded data/2019-2028-capital-budget-and-plan-details.xlsx
Downloaded data/2020-to-2029-capital-budget-plan-by-ward.xlsx
Downloaded data/2021-2030-capital-budget-and-plan-details.xlsx
Downloaded data/2022-2031-capital-budget-and-plan-details.xlsx
Downloaded data/2023-2032-capital-budget-and-plan-details.xlsx
Downloaded data/2024-2033-capital-budget-and-